In [1]:
import os
import re
from dotenv import load_dotenv
import pandas as pd

import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

from typing import List, Any, Optional, Dict
from rich.pretty import pprint

def pretty_print(title: str=None, content: Any=None):
    if title is None:
        print(content)
        return
    print(title)
    pprint(content)

from IPython.display import display, HTML, Markdown

In [2]:
from typing import Any, Optional, List
from rich.pretty import pprint

def pretty_print(title: Optional[str] = None, content: Any = None, exclude: Optional[List[str]] = None):
    """
    Prints the content prettily. If a title is provided, it prints the title before the content.
    Can exclude specified attributes from being printed if content is a dictionary.

    Args:
        title (Optional[str]): The title to print before the content. Defaults to None.
        content (Any): The content to be printed.
        exclude (Optional[List[str]]): List of keys to exclude from the content if it's a dictionary. Defaults to None.
    """
    if exclude is not None and isinstance(content, dict):
        content = {k: v for k, v in content.items() if k not in exclude}
    
    if title is not None:
        print(title)
    
    pprint(content)

In [3]:
load_dotenv()
os.chdir(os.path.dirname(os.getcwd()))

In [4]:
df = pd.read_parquet("./data/splade_embeds.parquet")
# Convert column names to snake_case for compatibility with LanceDB
original_columns = df.columns
snake_case_columns = {col: re.sub(r'(?<!^)(?=[A-Z])', '_', col).lower() for col in original_columns}
df.rename(columns=snake_case_columns, inplace=True)
df.head(1)

,index,created_utc,full_link,id,body,title,text_label,flair_label,embeddings,token_count,llm_title,state,kmeans_label,topic_title,splade_embeddings
0,1078,1575952538,https://www.reddit.com/r/legaladvice/comments/...,e8lsen,I applied for a job and after two interviews I...,"Failed a drug test due to amphetamines, I have...",employment,5,"[9.475638042064453e-05, 0.0005111666301983955,...",493,"""Validity of Schedule II Drug Prescription in ...",PR,8,Employment Legal Concerns and Issues,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [5]:
from llama_index.core.schema import TextNode, BaseNode

def dataframe_to_text_nodes(
    df: pd.DataFrame, 
    text_col: str,
    metadata_fields: List[str], 
    embedding_column: Optional[List[float]] = []
) -> List[TextNode]:
    """
    Creates a list of TextNode objects from a DataFrame based on specified fields for text, metadata, excluded metadata keys, and optionally embedding columns.

    Args:
        df (pd.DataFrame): The DataFrame containing the data to be converted into TextNode objects.
        text_col (str): The column name in the DataFrame to use as the text for each TextNode.
        metadata_fields (List[str]): A list of column names in the DataFrame to include in the metadata of each TextNode.
        excluded_metadata_keys (List[str] | None): A list of keys to exclude from the embedded metadata in the text representation of each TextNode. Defaults to None.
        embedding_column (Optional[List[str]]): A list of column names whose values should be added as an embedding attribute to each TextNode. If None, the embedding attribute is skipped. Defaults to None.

    Returns:
        List[TextNode]: A list of TextNode objects created from the DataFrame.
    """
    nodes = []
    for index, row in df.iterrows():
        metadata = {field: row[field] for field in metadata_fields}
        if embedding_column:
            embedding = row[embedding_column].tolist()
        node = TextNode(
            id_=index,
            text=row[text_col],
            embedding=embedding or None,
            metadata=metadata,
            metadata_template="{key} = {value}",
            text_template="Metadata:\n{metadata_str}\n----------------------------------------\nContent:\n{content}",
            )
        nodes.append(node)
    return nodes

In [6]:
df.rename(columns={'llm_title': 'title'}, inplace=True)
nodes = dataframe_to_text_nodes(df=df, text_col='body', metadata_fields=['state', 'full_link', 'title'], embedding_column='embeddings')

In [7]:
pretty_print(nodes[1])

Node ID: 1
Text: Hi everyone, thanks in advance for any guidance.  I was driving
home yesterday and managed to get pulled over outside Chattanooga
going 21 miles over the speed limit (76 in a 55).  It was unfortunate,
right at the part of the highway where the speed limit drops from 70
to 55 because of the mountains, and I just didn't adjust in time.  I
usually ...


None

In [8]:
from typing import Tuple
import numpy as np
from typing import cast
from llama_index.core.vector_stores.types import VectorStore
from llama_index.core.vector_stores import MetadataFilters, ExactMatchFilter
from llama_index.core.vector_stores import (
    VectorStoreQuery,
    VectorStoreQueryResult,
)
from llama_index.core.schema import BaseNode


def get_top_k_embeddings(
    query_embedding: List[float],
    doc_embeddings: List[List[float]],
    doc_ids: List[str],
    similarity_top_k: int = 5,
) -> Tuple[List[float], List]:
    """Get top nodes by similarity to the query."""
    # dimensions: D
    qembed_np = np.array(query_embedding)
    # dimensions: N x D
    dembed_np = np.array(doc_embeddings)
    # dimensions: N
    dproduct_arr = np.dot(dembed_np, qembed_np)
    # dimensions: N
    norm_arr = np.linalg.norm(qembed_np) * np.linalg.norm(
        dembed_np, axis=1, keepdims=False
    )
    # dimensions: N
    cos_sim_arr = dproduct_arr / norm_arr

    # now we have the N cosine similarities for each document
    # sort by top k cosine similarity, and return ids
    tups = [(cos_sim_arr[i], doc_ids[i]) for i in range(len(doc_ids))]
    sorted_tups = sorted(tups, key=lambda t: t[0], reverse=True)

    sorted_tups = sorted_tups[:similarity_top_k]

    result_similarities = [s for s, _ in sorted_tups]
    result_ids = [n for _, n in sorted_tups]
    return result_similarities, result_ids


def filter_nodes(nodes: List[BaseNode], filters: MetadataFilters):
    filtered_nodes = []
    for node in nodes:
        matches = True
        for f in filters.filters:
            if f.key not in node.metadata:
                matches = False
                continue
            if f.value != node.metadata[f.key]:
                matches = False
                continue
        if matches:
            filtered_nodes.append(node)
    return filtered_nodes


def dense_search(query: VectorStoreQuery, nodes: List[BaseNode]):
    """Dense search."""
    query_embedding = cast(List[float], query.query_embedding)
    doc_embeddings = [n.embedding for n in nodes]
    doc_ids = [n.node_id for n in nodes]
    return get_top_k_embeddings(
        query_embedding,
        doc_embeddings,
        doc_ids,
        similarity_top_k=query.similarity_top_k,
    )

In [9]:
class BaseVectorStore(VectorStore):
    """Simple custom Vector Store.

    Stores documents in a simple in-memory dict.

    """

    stores_text: bool = True
    
    def __init__(self) -> None:
        """Init params."""
        self.node_dict: Dict[str, BaseNode] = {}

    def get(self, text_id: str) -> List[float]:
        """Get embedding."""
        return self.node_dict[text_id]

    def add(
        self,
        nodes: List[BaseNode],
    ) -> List[str]:
        """Add nodes to index."""
        for node in nodes:
            self.node_dict[node.node_id] = node

    def delete(self, node_id: str, **delete_kwargs: Any) -> None:
        """
        Delete nodes using with node_id.

        Args:
            node_id: str

        """
        del self.node_dict[node_id]

    def query(
        self,
        query: VectorStoreQuery,
        **kwargs: Any,
    ) -> VectorStoreQueryResult:
        """Get nodes for response."""

        query_embedding = cast(List[float], query.query_embedding)
        doc_embeddings = [n.embedding for n in self.node_dict.values()]
        doc_ids = [n.node_id for n in self.node_dict.values()]

        similarities, node_ids = get_top_k_embeddings(
            query_embedding,
            doc_embeddings,
            doc_ids,
            similarity_top_k=query.similarity_top_k,
        )
        result_nodes = [self.node_dict[node_id] for node_id in node_ids]

        return VectorStoreQueryResult(
            nodes=result_nodes, similarities=similarities, ids=node_ids
        )

    def persist(self, persist_path, fs=None) -> None:
        """Persist the SimpleVectorStore to a directory.

        NOTE: we are not implementing this for now.

        """
        pass

In [10]:
vector_store = BaseVectorStore()
vector_store.add(nodes)

In [11]:
print(str(nodes[0]))

Node ID: 0
Text: I applied for a job and after two interviews I was given for and
agreed to an offer letter, had a start date.  The only remaining
variable was a drug test, which I prepared myself for by bringing all
my prescriptions with me.    I take Adderall, my doctor prescribes me
for 3 pills a day, insurance pays for only two a day so that’s what I
get.  2...


In [12]:
import openai

client = openai.OpenAI()

In [13]:
query_str = "Can you tell me about madicinal marijuana use at work?"

query_embedding = client.embeddings.create(
                    input=query_str, model='text-embedding-ada-002'
                ).data[0].embedding

In [14]:
query_obj = VectorStoreQuery(
    query_embedding=query_embedding, similarity_top_k=5
)

query_result = vector_store.query(query_obj)
for similarity, node in zip(query_result.similarities, query_result.nodes):
    print(
        "\n----------------\n"
        f"[Node ID {node.node_id}] Similarity: {similarity}\n\n"
        f"{node.get_content(metadata_mode='all')}"
        "\n----------------\n\n"
    )


----------------
[Node ID 2172] Similarity: 0.851869304515034

Metadata:
state = MI
full_link = https://www.reddit.com/r/legaladvice/comments/gr4q8s/is_it_legal_in_new_york_to_discriminate_against/
title = title    Is it legal in New York to discriminate agains...
title    "Can a hospital refuse to hire a medical marij...
Name: 2172, dtype: object
----------------------------------------
Content:
I'm a New York State medical marijuana patient. I also work in healthcare. I applied to a new job at a new hospital, and they are discriminating against me for being a medical marijuana patient. I was offered the job and accepted, but when I went to get my pre-employment physical conducted, I gave them my medical marijuana card and informed them that I am a patient. They are now refusing to hire me. Is this legal? I already contacted the division of human rights at the labor department and they said I may or may not have a case.
----------------



----------------
[Node ID 3185] Similarity: 

In [15]:
filters = MetadataFilters(
    filters=[
        ExactMatchFilter(key="page", value=3)
    ]
)

query_obj = VectorStoreQuery(
    query_embedding=query_embedding, similarity_top_k=3, filters=filters
)

query_result = vector_store.query(query_obj)
for similarity, node in zip(query_result.similarities, query_result.nodes):
    print(
        "\n----------------\n"
        f"[Node ID {node.node_id}] Similarity: {similarity}\n\n"
        f"{node.get_content(metadata_mode='all')}"
        "\n----------------\n\n"
    )


----------------
[Node ID 2172] Similarity: 0.851869304515034

Metadata:
state = MI
full_link = https://www.reddit.com/r/legaladvice/comments/gr4q8s/is_it_legal_in_new_york_to_discriminate_against/
title = title    Is it legal in New York to discriminate agains...
title    "Can a hospital refuse to hire a medical marij...
Name: 2172, dtype: object
----------------------------------------
Content:
I'm a New York State medical marijuana patient. I also work in healthcare. I applied to a new job at a new hospital, and they are discriminating against me for being a medical marijuana patient. I was offered the job and accepted, but when I went to get my pre-employment physical conducted, I gave them my medical marijuana card and informed them that I am a patient. They are now refusing to hire me. Is this legal? I already contacted the division of human rights at the labor department and they said I may or may not have a case.
----------------



----------------
[Node ID 3185] Similarity: 

In [16]:
from llama_index.core import VectorStoreIndex

vector_index = VectorStoreIndex.from_vector_store(vector_store)
query_engine = vector_index.as_query_engine()

response = query_engine.query("Write a detailed summary of the legal issues involving medical marijuana")
Markdown(str(response))

The legal issues involving medical marijuana can vary depending on the state and the specific circumstances. In the first scenario, a mobile home park in California is restricting a resident from using medical marijuana on the park's land, despite the resident owning the home. This raises questions about the resident's legal rights as a homeowner and a medical marijuana patient within the confines of the park's regulations.

In the second scenario, a healthcare worker in New York is facing discrimination from a hospital for being a medical marijuana patient. The hospital's refusal to hire the individual based on their medical marijuana card brings up concerns about employment discrimination and the rights of medical marijuana patients in the workplace.

Overall, the legal issues surrounding medical marijuana often involve conflicts between state laws permitting medical marijuana use and federal laws that still classify marijuana as a controlled substance. These conflicts can impact various aspects of life, including housing rights, employment opportunities, healthcare access, and more.

In [17]:
Markdown(response.source_nodes[0].get_content())

Asking this for a friend who lives in a mobile home park in California.  He wants to get a medical marijuana card for his medical conditions so he can get off his pain medicine, but the park is telling him that even though he owns his home 100%, because its on the park's land he can't use it or bring it on the property.  Can the park legally do this or is there some legal medical grounds for him?  

In [18]:
TEMPERATURE = 0.0
SIM_TOP_K = 6
RERANK_TOP_K = 3
WIN_SZ = 1024

import asyncio
from tqdm.asyncio import tqdm

import chromadb
from chromadb.api.models.Collection import Collection

from llama_index.core import (
    ServiceContext,
    SimpleDirectoryReader,
    StorageContext,
    VectorStoreIndex,
    get_response_synthesizer,
)
from llama_index.core import QueryBundle
from llama_index.llms.openai import OpenAI
from llama_index.core import PromptTemplate
from llama_index.core.llms.utils import LLMType
from llama_index.core.schema import NodeWithScore
from llama_index.core.indices.base import BaseIndex
from llama_index.core.retrievers import BaseRetriever
from llama_index.core.embeddings.utils import EmbedType
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.legacy.vector_stores import ChromaVectorStore
from llama_index.core.base.llms.types import CompletionResponse
from llama_index.core.node_parser import SentenceWindowNodeParser

from llama_index.core.retrievers import BaseRetriever
from llama_index.core import get_response_synthesizer
from llama_index.legacy.core.response.schema import RESPONSE_TYPE
from llama_index.core.response_synthesizers import BaseSynthesizer
from llama_index.core.query_engine import CustomQueryEngine, BaseQueryEngine

In [19]:
li_model = OpenAI(model="gpt-4-1106-preview", temperature=TEMPERATURE)

In [20]:
vector_index = VectorStoreIndex.from_vector_store(
    vector_store=vector_store,
)

In [21]:
base_retriever = vector_index.as_retriever(similarity_top_k=10)

In [23]:
from collections import defaultdict


class MultiQueriesRetriever(BaseRetriever):
    def __init__(self, base_retriever: BaseRetriever, model:OpenAI):
        self.template = PromptTemplate("""You are an AI language model assistant. Your task is to generate Five
    different versions of the given user question to retrieve relevant documents from a vector
    database. By generating multiple perspectives on the user question, your goal is to help
    the user overcome some of the limitations of the distance-based similarity search.
    Provide these alternative questions seperated by newlines only.

    For example, these alternative questions:

    'What is Bill Gates known for?'
│   "Can you provide information about Bill Gates' background?"

    Not:

    '1. What is Bill Gates known for?'
│   "2. Can you provide information about Bill Gates' background?"

    Original question: {question}""")
        self._retrievers = [base_retriever]
        self.base_retriever = base_retriever
        self.model = model

    @classmethod
    def flatten(cls, lst: List[List[Any]]) -> List[Any]:
        return [element for sublist in lst for element in sublist]

    def gen_queries(self, query: str) -> List[str]:
        gen_queries_model = OpenAI(model="gpt-3.5-turbo-0125", temperature=1.5)
        prompt = self.template.format(question=query)
        res = gen_queries_model.complete(prompt)
        return res.text.split("\n")

    async def run_gen_queries(self,generated_queries: List[str]) -> List[NodeWithScore]:
        tasks = list(map(lambda q: self.base_retriever.aretrieve(q), generated_queries))
        res = await tqdm.gather(*tasks)
        return MultiQueriesRetriever.flatten(res)

    def _retrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
        return list()

    async def _aretrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
        query: str= query_bundle.query_str
        generated_queries: List[str] = self.gen_queries(query)
        pretty_print("generated_queries", generated_queries)
        node_with_scores = await self.run_gen_queries(generated_queries)
        # node_with_scores_uniqued = dict()
        # node_with_scores_uniqued = {node_with_score.get_content(): node_with_score for node_with_score in node_with_scores}
        # return node_with_scores_uniqued.values()
        # Fusion with ranking, ref:https://plg.uwaterloo.ca/~gvcormac/cormacksigir09-rrf.pdf
        output_documents = self._reciprocal_rank_fusion(node_with_scores)
        return output_documents
     
    def _reciprocal_rank_fusion(self, document_lists: List[List[NodeWithScore]]) -> List[NodeWithScore]:
        """
        Merge multiple lists of NodeWithScore and assign scores based on reciprocal rank fusion.
        
        Args:
            document_lists (List[List[NodeWithScore]]): A list of lists, where each sublist contains NodeWithScore objects.
        
        Returns:
            List[NodeWithScore]: A list of NodeWithScore objects with updated scores based on reciprocal rank fusion.
        
        The constant k is set to 61 (60 was suggested by the original paper,
        plus 1 as python lists are 0-based and the paper used 1-based ranking).
        """
        k = 61

        scores_map = defaultdict(int)
        documents_map = {}
        weights = self.weights if self.weights else [1 / len(document_lists)] * len(document_lists)

        # Calculate weighted reciprocal rank fusion score
        for documents, weight in zip(document_lists, weights):
            for rank, doc in enumerate(documents):
                scores_map[doc.id] += (weight * len(document_lists)) / (k + rank)
                documents_map[doc.id] = doc

        # Normalize scores. Note: len(results) / k is the maximum possible score,
        # achieved by being ranked first in all doc lists with non-zero weight.
        for id in scores_map:
            scores_map[id] /= len(document_lists) / k

        for doc in documents_map.values():
            doc.score = scores_map[doc.id]

        return documents_map.values()



In [24]:
mr = MultiQueriesRetriever(base_retriever, li_model)

In [25]:
import nest_asyncio
nest_asyncio.apply()

ls = mr.gen_queries("Is medical marijuana allowed at work in California?")
pretty_print("ls", ls)
rls = await mr.run_gen_queries(ls)

ls


[
│   'What are the regulations around medical marijuana use in the workplace in California?',
│   'Are employees in California allowed to use medical marijuana at their jobs?',
│   'What is the policy on medical marijuana use in the workplace in California?',
│   'Is medical marijuana use permitted on the job in California?',
│   'Does California law allow for medical marijuana use during work hours?'
]

100%|██████████| 5/5 [00:01<00:00,  3.55it/s]


In [91]:
rls[0]

NodeWithScore(node=TextNode(id_='1764', embedding=[-0.01891859671613266, -0.009167221620984913, 0.008576430610654549, -0.05597249568804771, -0.029898022173049425, 0.020193113556364772, -0.021095894843601734, -0.012200835488550457, -0.027017083744530816, 0.00011907099862192289, 0.008549878603339238, -0.02193229704387302, 0.00874238298468174, 0.0016570367596849447, 0.00797900113283406, 0.02529117719200282, 0.043147676851716256, -0.021892468101577457, 0.015732307409520176, -0.012174283481235147, -0.015480059614734325, -0.0038102719559015325, -0.02011345567177364, 0.013203189131526336, -0.013037236757499144, 0.03125219503522747, 0.01891859671613266, -0.03220808219974025, -0.013833809084152264, -0.02282180325877493, 0.03757166963382181, 0.013740876127226076, -0.023963556337140087, -0.020432085347492968, -0.034305722442618196, -0.015904898251037497, 0.010700623792170403, -0.011317967741138676, 0.008204696920304599, -0.0109395955832986, 0.022556277597686223, -0.017259071113215542, 0.005300526

In [26]:
class MultiQueriers:
    def __init__(self,
                 base_retriever: BaseRetriever,
                 base_query_engine: BaseQueryEngine,
                 model:OpenAI,
                 sub_queries_in_bundle_to_answer: bool=True):
        self.base_retriever = base_retriever
        self.base_query_engine = base_query_engine
        self.model = model
        self.sub_queries_in_bundle_to_answer = sub_queries_in_bundle_to_answer
        self.gen_q_template = PromptTemplate("""You are an AI language model assistant. Your task is to generate Five
    different versions of the given user question to retrieve relevant documents from a vector
    database. By generating multiple perspectives on the user question, your goal is to help
    the user overcome some of the limitations of the distance-based similarity search.
    Provide these alternative questions seperated by newlines only.

    For example, these alternative questions:

    'What is Bill Gates known for?'
│   "Can you provide information about Bill Gates' background?"

    Not:

    '1. What is Bill Gates known for?'
│   "2. Can you provide information about Bill Gates' background?"

    Original question: {question}""")
        self.qa_prompt_template = PromptTemplate("""Here is the question you need to answer:

    \n --- \n {query_str} \n --- \n

    Here is any available background question + answer pairs:

    \n --- \n {q_a_pairs} \n --- \n

    Here is additional context relevant to the question:

    \n --- \n {context_str} \n --- \n

    Use the above context and any background question + answer pairs to answer the question: \n {query_str}
    """)

    def gen_queries(self, query: str) -> List[str]:
        gen_queries_model = OpenAI(model="gpt-3.5-turbo-0125", temperature=1.5)
        prompt = self.gen_q_template.format(question=query)
        res = gen_queries_model.complete(prompt)
        return res.text.split("\n")

    def query_by_retriever(self, query_str: str) ->str:
        nodes = self.base_retriever.retrieve(query_str)
        res = "\n\n".join([n.node.get_content() for n in nodes])
        return res

    async def run_gen_queries(self,generated_queries: List[str]) -> str:
        sub_query_qa_pairs = list()
        if self.sub_queries_in_bundle_to_answer:
            # Answer all queries in one bundle.
            tasks = list(map(lambda q: self.base_query_engine.aquery(q), generated_queries))
            res = await tqdm.gather(*tasks)
            for idx, (query, answer) in enumerate(zip(generated_queries, res)):
                qa_pair = f"Question {idx}: {query}\nAnswer: {answer}\n"
                sub_query_qa_pairs.append(qa_pair)
            pretty_print("sub_query_qa_pairs", sub_query_qa_pairs)
            return "\n\n".join(sub_query_qa_pairs)
        else:
            # Answer queries in step-wise.
            # One sub-query will be answered based on the context of sub-query,
            # history of pairs of previous queries and answers.
            for idx, query in enumerate(generated_queries):
                pretty_print(f"{idx}. query", query)
                pretty_print(f"{idx}. sub_query_qa_pairs", sub_query_qa_pairs)
                context_str = self.query_by_retriever(query)
                sub_query = self.qa_prompt_template.format(
                                  query_str=query,
                                  q_a_pairs="\n\n".join(sub_query_qa_pairs),
                                  context_str=context_str
                )
                pretty_print("sub_query", sub_query)
                answer: str = self.model.complete(sub_query)
                qa_pair = f"Question {idx}: {query}\nAnswer: {answer}\n"
                sub_query_qa_pairs.append(qa_pair)
            return "\n\n".join(sub_query_qa_pairs)

    def query(self, query_str: str) -> CompletionResponse:
        return ""

    async def aquery(self, query_str: str) -> CompletionResponse:
        generated_queries: List[str] = self.gen_queries(query_str)
        sub_query_qa_pairs: str = await self.run_gen_queries(generated_queries)
        context_str = self.query_by_retriever(query_str)
        final_query: str = self.qa_prompt_template.format(query_str=query_str,
                                q_a_pairs=sub_query_qa_pairs,
                                context_str=context_str)
        pretty_print("final_query", final_query)
        response: str = self.model.complete(final_query)
        return response

In [29]:
import nest_asyncio
nest_asyncio.apply()

query_text = """What are the legal issues involving medical marijuana and employment?

    The response should be presented as a list of key points, after creating the title of the content,
    formatted in Markdown with appropriate markup for clarity and organization.
    """
mqe=MultiQueriers(base_retriever,  vector_index.as_query_engine(), li_model, sub_queries_in_bundle_to_answer=True)
final_res = await mqe.aquery(query_text)
display(Markdown(final_res.text))

100%|██████████| 5/5 [00:04<00:00,  1.10it/s]

sub_query_qa_pairs


[
│   'Question 0: What are the implications of medical marijuana on employment laws?\nAnswer: Employment laws regarding medical marijuana can vary from state to state. It is important to be aware of the specific regulations in the state where you are seeking employment. Issues such as drug testing policies, accommodations for medical marijuana patients, and potential discrimination based on medical marijuana use can impact employment situations. It is advisable to understand the legal framework in your state and seek guidance if you have concerns about how medical marijuana may affect your employment.\n',
│   'Question 1: What legal challenges arise from the intersection of medical marijuana and workforce policies?\nAnswer: Legal challenges that arise from the intersection of medical marijuana and workforce policies include issues related to discrimination against medical marijuana patients in hiring practices, conflicts between state laws legalizing medical marijuana and company drug policies, and concerns about privacy and disclosure of medical marijuana cardholder status in the workplace. Additionally, questions may arise regarding the extent to which employers can enforce zero-tolerance drug policies in states where medical marijuana is legal, and the potential violation of privacy rights such as HIPAA if medical cardholder information is disclosed during drug testing processes.\n',
│   "Question 2: How do regulations treat medical marijuana use in the context of workplace relationships?\nAnswer: Regulations regarding medical marijuana use in the context of workplace relationships can vary depending on the state and the specific company policies in place. In some cases, companies with zero tolerance drug policies may still prohibit employees from using medical marijuana, even if they have a valid medical card. This can lead to conflicts between state laws legalizing medical marijuana and company policies that prohibit its use. Additionally, there may be instances where discrimination against medical marijuana patients in the hiring process could be considered illegal under certain state laws. It is important for individuals to be aware of both state regulations regarding medical marijuana and their company's specific drug policies to understand how their medical marijuana use may be treated in the workplace.\n",
│   'Question 3: Are there any specific legal cases or precedents related to medical marijuana and employment rights?\nAnswer: There have been legal cases and precedents related to medical marijuana and employment rights in various states.\n',
│   "Question 4: What are the legal considerations for employers with regards to medical marijuana use by employees?\nAnswer: Employers need to consider the legal implications of their drug policies when it comes to employees using medical marijuana. While some states have legalized medical marijuana, employers may still enforce zero tolerance drug policies. It is important for employers to be aware of state laws regarding medical marijuana and to ensure that their policies comply with both state and federal regulations. Additionally, employers should handle information about employees' medical marijuana use in a confidential manner to avoid potential violations of privacy laws like HIPAA.\n"
]

final_query


'Here is the question you need to answer:\n\n    \n --- \n What are the legal issues involving medical marijuana and employment?\n\n    The response should be presented as a list of key points, after creating the title of the content,\n    formatted in Markdown with appropriate markup for clarity and organization.\n     \n --- \n\n\n    Here is any available background question + answer pairs:\n\n    \n --- \n Question 0: What are the implications of medical marijuana on employment laws?\nAnswer: Employment laws regarding medical marijuana can vary from state to state. It is important to be aware of the specific regulations in the state where you are seeking employment. Issues such as drug testing policies, accommodations for medical marijuana patients, and potential discrimination based on medical marijuana use can impact employment situations. It is advisable to understand the legal framework in your state and seek guidance if you have concerns about how medical marijuana may affect your employment.\n\n\nQuestion 1: What legal challenges arise from the intersection of medical marijuana and workforce policies?\nAnswer: Legal challenges that arise from the intersection of medical marijuana and workforce policies include issues related to discrimination against medical marijuana patients in hiring practices, conflicts between state laws legalizing medical marijuana and company drug policies, and concerns about privacy and disclosure of medical marijuana cardholder status in the workplace. Additionally, questions may arise regarding the extent to which employers can enforce zero-tolerance drug policies in states where medical marijuana is legal, and the potential violation of privacy rights such as HIPAA if medical cardholder information is disclosed during drug testing processes.\n\n\nQuestion 2: How do regulations treat medical marijuana use in the context of workplace relationships?\nAnswer: Regulations regarding medical marijuana use in the context of workplace relationships can vary depending on the state and the specific company policies in place. In some cases, companies with zero tolerance drug policies may still prohibit employees from using medical marijuana, even if they have a valid medical card. This can lead to conflicts between state laws legalizing medical marijuana and company policies that prohibit its use. Additionally, there may be instances where discrimination against medical marijuana patients in the hiring process could be considered illegal under certain state laws. It is important for individuals to be aware of both state regulations regarding medical marijuana and their company\'s specific drug policies to understand how their medical marijuana use may be treated in the workplace.\n\n\nQuestion 3: Are there any specific legal cases or precedents related to medical marijuana and employment rights?\nAnswer: There have been legal cases and precedents related to medical marijuana and employment rights in various states.\n\n\nQuestion 4: What are the legal considerations for employers with regards to medical marijuana use by employees?\nAnswer: Employers need to consider the legal implications of their drug policies when it comes to employees using medical marijuana. While some states have legalized medical marijuana, employers may still enforce zero tolerance drug policies. It is important for employers to be aware of state laws regarding medical marijuana and to ensure that their policies comply with both state and federal regulations. Additionally, employers should handle information about employees\' medical marijuana use in a confidential manner to avoid potential violations of privacy laws like HIPAA.\n \n --- \n\n\n    Here is additional context relevant to the question:\n\n    \n --- \n I\'m a New York State medical marijuana patient. I also work in healthcare. I applied to a new job at a new hospital, and they are discriminating against me for being a medical marijuana patient. I was offered the jo

# Key Legal Issues Involving Medical Marijuana and Employment

Medical marijuana and employment law is a complex and evolving area. Below are key points that highlight the legal issues surrounding this topic:

1. **State vs. Federal Law Conflicts**: Marijuana remains illegal under federal law, classified as a Schedule I substance. However, many states have legalized medical marijuana, creating a conflict between state and federal laws that complicates employment policies and protections.

2. **Workplace Drug Policies**: Employers often have drug-free workplace policies that include zero-tolerance for drug use. These policies may conflict with state laws that protect medical marijuana patients, leading to legal challenges.

3. **Discrimination Concerns**: Employees with medical marijuana prescriptions may face discrimination in hiring, firing, or other employment decisions. Some states have laws protecting medical marijuana patients from employment discrimination, but these protections vary widely.

4. **Safety-Sensitive Positions**: Employers may have more leeway to prohibit medical marijuana use for safety-sensitive positions, such as those involving operation of heavy machinery or driving, where impairment could pose significant risks.

5. **Drug Testing**: Employers may conduct drug testing, but the legality of taking adverse employment actions against medical marijuana users based on test results can be legally complex, especially in states with protections for medical marijuana users.

6. **Accommodation Requirements**: Under certain state laws, employers may be required to provide reasonable accommodations for medical marijuana users, as long as it does not cause undue hardship or compromise workplace safety.

7. **Privacy Issues**: Disclosure of an employee's medical marijuana cardholder status during drug testing could raise privacy concerns, potentially implicating laws like the Health Insurance Portability and Accountability Act (HIPAA).

8. **Workers' Compensation and Insurance**: The use of medical marijuana may affect workers' compensation claims or employer-provided insurance coverage, with some insurers denying coverage for medical marijuana-related treatments.

9. **Legal Precedents**: There have been legal cases that set precedents regarding medical marijuana and employment rights, but these vary by state and the specific circumstances of each case.

10. **Property Restrictions**: Even if an employee legally uses medical marijuana, restrictions may be in place regarding its use on certain properties, such as rented spaces or employer-owned facilities.

11. **Federal Employees and Contractors**: Federal employees and contractors are subject to federal drug-free workplace laws, which prohibit marijuana use regardless of state laws or medical marijuana status.

12. **Impact on Unemployment Benefits**: Testing positive for marijuana, even with a medical prescription, can impact eligibility for unemployment benefits in some states.

13. **Employer Education and Policy Updates**: Employers must stay informed about the changing legal landscape and may need to update their policies to comply with new state laws regarding medical marijuana.

14. **Potential for Legal Action**: Employees who feel they have been wrongfully discriminated against due to their medical marijuana use may have grounds for legal action, but outcomes can be uncertain and heavily dependent on state laws and judicial interpretations.

Understanding the legal framework in your state and seeking legal guidance is crucial for both employers and employees navigating the intersection of medical marijuana and employment law.

In [30]:
query_text = """What are the legal issues involving medical marijuana and employment?

    The response should be presented as a list of key points, after creating the title of the content,
    formatted in Markdown with appropriate markup for clarity and organization.
    """
mqe=MultiQueriers(base_retriever,  vector_index.as_query_engine(), li_model, sub_queries_in_bundle_to_answer=False)
final_res = await mqe.aquery(query_text)
display(Markdown(final_res.text))

0. query


'What legal concerns arise from the use of medical marijuana in the workplace?'

0. sub_query_qa_pairs


[]

sub_query


...

sub_query


'Here is the question you need to answer:\n\n    \n --- \n What legal challenges do individuals face with medical marijuana use in employment settings? \n --- \n\n\n    Here is any available background question + answer pairs:\n\n    \n --- \n Question 0: What legal concerns arise from the use of medical marijuana in the workplace?\nAnswer: The use of medical marijuana in the workplace raises several legal concerns, which can vary depending on the state laws, the nature of the job, and federal regulations. Here are some of the key legal issues that may arise:\n\n1. **State vs. Federal Law**: Marijuana remains illegal under federal law as a Schedule I controlled substance, which can create conflicts with state laws that have legalized medical marijuana. This is particularly relevant in workplaces with federal contracts or in industries that are heavily regulated by the federal government, such as transportation.\n\n2. **Drug-Free Workplace Policies**: Many employers maintain drug-free workplace policies that prohibit the use of marijuana, including medical marijuana. Employers may argue that such policies are necessary to maintain safety, productivity, or compliance with federal laws or contracts.\n\n3. **Disability Discrimination**: Employees who are medical marijuana patients may have underlying health conditions that qualify as disabilities under the Americans with Disabilities Act (ADA) or state disability laws. Employers may need to consider whether they are required to make reasonable accommodations for such employees, although the ADA does not require accommodations for the use of illegal drugs.\n\n4. **Safety-Sensitive Positions**: Employers may have legitimate concerns about the use of medical marijuana in safety-sensitive positions, where impairment could lead to accidents or injuries. This is particularly relevant in healthcare, transportation, and industries involving heavy machinery.\n\n5. **Drug Testing**: Legal issues can arise from drug testing for marijuana, especially since THC, the psychoactive component of marijuana, can remain in a person\'s system for days or weeks after use. This can lead to positive drug tests even if the employee is not impaired at work.\n\n6. **Privacy Concerns**: Disclosing one\'s status as a medical marijuana patient can raise privacy issues, and there may be concerns about whether such disclosure violates the Health Insurance Portability and Accountability Act (HIPAA) or other privacy laws.\n\n7. **Workers\' Compensation and Unemployment Benefits**: The use of medical marijuana may affect claims for workers\' compensation or unemployment benefits if an employee is injured or terminated due to a positive drug test.\n\n8. **State Protections for Medical Marijuana Users**: Some states have enacted laws that provide specific protections for medical marijuana patients, prohibiting discrimination based on an individual\'s status as a medical marijuana patient. Employers in these states may need to navigate these protections when making employment decisions.\n\n9. **Contractual and Collective Bargaining Agreements**: Unionized workplaces may have collective bargaining agreements that address drug use and testing, which could impact how medical marijuana use is handled.\n\n10. **Housing and Land Use**: For individuals living in certain communities, such as mobile home parks or rental properties, there may be rules or lease agreements that restrict the use of marijuana on the premises, even if it is for medical purposes.\n\nIn summary, the legal landscape surrounding medical marijuana in the workplace is complex and evolving. Employers and employees must navigate a patchwork of federal, state, and local laws, as well as company policies, while considering the rights and safety of all parties involved. Legal advice is often necessary to address specific situations, such as the case of the New York State medical marijuana patient facing potential discrimination in a healthcare job.\n\n\nQuesti

final_query


'Here is the question you need to answer:\n\n    \n --- \n What are the legal issues involving medical marijuana and employment?\n\n    The response should be presented as a list of key points, after creating the title of the content,\n    formatted in Markdown with appropriate markup for clarity and organization.\n     \n --- \n\n\n    Here is any available background question + answer pairs:\n\n    \n --- \n Question 0: What legal concerns arise from the use of medical marijuana in the workplace?\nAnswer: The use of medical marijuana in the workplace raises several legal concerns, which can vary depending on the state laws, the nature of the job, and federal regulations. Here are some of the key legal issues that may arise:\n\n1. **State vs. Federal Law**: Marijuana remains illegal under federal law as a Schedule I controlled substance, which can create conflicts with state laws that have legalized medical marijuana. This is particularly relevant in workplaces with federal contracts or in industries that are heavily regulated by the federal government, such as transportation.\n\n2. **Drug-Free Workplace Policies**: Many employers maintain drug-free workplace policies that prohibit the use of marijuana, including medical marijuana. Employers may argue that such policies are necessary to maintain safety, productivity, or compliance with federal laws or contracts.\n\n3. **Disability Discrimination**: Employees who are medical marijuana patients may have underlying health conditions that qualify as disabilities under the Americans with Disabilities Act (ADA) or state disability laws. Employers may need to consider whether they are required to make reasonable accommodations for such employees, although the ADA does not require accommodations for the use of illegal drugs.\n\n4. **Safety-Sensitive Positions**: Employers may have legitimate concerns about the use of medical marijuana in safety-sensitive positions, where impairment could lead to accidents or injuries. This is particularly relevant in healthcare, transportation, and industries involving heavy machinery.\n\n5. **Drug Testing**: Legal issues can arise from drug testing for marijuana, especially since THC, the psychoactive component of marijuana, can remain in a person\'s system for days or weeks after use. This can lead to positive drug tests even if the employee is not impaired at work.\n\n6. **Privacy Concerns**: Disclosing one\'s status as a medical marijuana patient can raise privacy issues, and there may be concerns about whether such disclosure violates the Health Insurance Portability and Accountability Act (HIPAA) or other privacy laws.\n\n7. **Workers\' Compensation and Unemployment Benefits**: The use of medical marijuana may affect claims for workers\' compensation or unemployment benefits if an employee is injured or terminated due to a positive drug test.\n\n8. **State Protections for Medical Marijuana Users**: Some states have enacted laws that provide specific protections for medical marijuana patients, prohibiting discrimination based on an individual\'s status as a medical marijuana patient. Employers in these states may need to navigate these protections when making employment decisions.\n\n9. **Contractual and Collective Bargaining Agreements**: Unionized workplaces may have collective bargaining agreements that address drug use and testing, which could impact how medical marijuana use is handled.\n\n10. **Housing and Land Use**: For individuals living in certain communities, such as mobile home parks or rental properties, there may be rules or lease agreements that restrict the use of marijuana on the premises, even if it is for medical purposes.\n\nIn summary, the legal landscape surrounding medical marijuana in the workplace is complex and evolving. Employers and employees must navigate a patchwork of federal, state, and local laws, as well as company policies, while considering the rights and safety of all parties involved. Legal advice is often n

# Legal Issues Involving Medical Marijuana and Employment

The intersection of medical marijuana use and employment law presents a variety of legal issues for both employees and employers. Below is a list of key points that outline these legal challenges:

- **State vs. Federal Law Conflicts**: Marijuana is classified as a Schedule I controlled substance under federal law, making it illegal at the federal level. However, many states have legalized medical marijuana, creating a conflict between state and federal laws that affects employment practices, especially in federally regulated industries or those with federal contracts.

- **Drug-Free Workplace Policies**: Employers may enforce drug-free workplace policies that prohibit the use of marijuana, including medical marijuana. These policies can lead to legal issues when they conflict with state laws that protect medical marijuana users.

- **Disability Discrimination**: Employees who use medical marijuana for disability-related reasons may face legal issues related to discrimination. While the Americans with Disabilities Act (ADA) does not protect the use of federally illegal drugs, some state laws may require employers to provide reasonable accommodations, excluding the use during work hours or impairment on the job.

- **Safety-Sensitive Positions**: Employers have a legitimate concern about impairment in safety-sensitive positions. Legal issues arise when employees with valid medical marijuana prescriptions are denied employment or disciplined due to their use, even if it is outside of work hours.

- **Drug Testing**: THC, the psychoactive component of marijuana, can remain in the body for a prolonged period, leading to positive drug tests. This can result in legal disputes over employment consequences for medical marijuana users who are not impaired at work.

- **Privacy Concerns**: Disclosing one's status as a medical marijuana patient can raise privacy issues. Legal challenges may occur if employees feel their privacy rights are violated or if there are concerns about the confidentiality of their medical information.

- **Workers' Compensation and Unemployment Benefits**: The use of medical marijuana can affect claims for workers' compensation or unemployment benefits, especially if an employee is injured or terminated due to a positive drug test for marijuana.

- **State Protections for Medical Marijuana Users**: Some states have enacted laws that protect medical marijuana patients from employment discrimination. Legal issues can arise when employers in these states take adverse actions against employees who are legally using medical marijuana.

- **Employer Accommodations**: Employers are not generally required to accommodate the use of medical marijuana, particularly if it would result in a violation of federal law or compromise safety standards. Legal disputes may occur when employees seek accommodations for their medical marijuana use.

- **Legal Ramifications of Failing a Drug Test**: Employees may face legal challenges if they fail a pre-employment or random drug test due to medical marijuana use. This can lead to not being hired or being terminated, depending on the employer's policies and the specific job role.

- **Housing and Land Use Restrictions**: For employees who use medical marijuana and live in certain communities, such as mobile home parks or rental properties, there may be rules or lease agreements that restrict the use of marijuana on the premises. This can indirectly affect employment if an employee requires the use of medical marijuana at home.

- **Impact on Federal Employees and Positions**: Federal employees or those in positions that require federal security clearances may face legal issues due to the strict prohibition of marijuana use under federal law, regardless of state laws permitting medical marijuana.

In summary, the legal landscape surrounding medical marijuana and employment is complex and evolving. Employers and employees must carefully navigate federal, state, and local laws, as well as company policies, to understand their rights and obligations. Legal advice is often necessary to address specific situations, and outcomes can vary based on jurisdiction and individual circumstances.